In [1]:
import sys
sys.path.append("/home/zixuan11/qjx/OpenKE")
import json
import os
import torch
torch.cuda.set_device(5)
torch.cuda.current_device()
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'
from typing import List, Optional
import torch
import seaborn as sns
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import copy

In [2]:
# import openke
from openke.config import Trainer, Tester
from openke.module.model import TransE
from openke.module.loss import MarginLoss
from openke.module.strategy import NegativeSampling
from openke.data import TrainDataLoader, TestDataLoader

In [3]:
# dataloader for training
train_dataloader = TrainDataLoader(
	in_path = "/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/", 
	nbatches = 100,
	threads = 8, 
	sampling_mode = "normal", 
	bern_flag = 1, 
	filter_flag = 1, 
	neg_ent = 25,
	neg_rel = 0)

Input Files Path : /home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/
The toolkit is importing datasets.
The total of relations is 237.
The total of entities is 14541.
The total of train triples is 272115.


In [5]:
train_dataloader

In [4]:
# dataloader for test
test_dataloader = TestDataLoader("/home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/", "link")

Input Files Path : /home/zixuan11/qjx/OpenKE/benchmarks/FB15K237/
The total of test triples is 20466.
The total of valid triples is 17535.


In [5]:
# define the model
transe = TransE(
	ent_tot = train_dataloader.get_ent_tot(),
	rel_tot = train_dataloader.get_rel_tot(),
	dim = 100, 
	p_norm = 1, 
	norm_flag = True)

In [6]:
# define the loss function
model = NegativeSampling(
	model = transe, 
	loss = MarginLoss(margin = 5.0),
	batch_size = train_dataloader.get_batch_size()
)

In [7]:
# train the model
trainer = Trainer(model = model, data_loader = train_dataloader, train_times = 10, alpha = 1.0, use_gpu = True)
trainer.run()
transe.save_checkpoint('/home/zixuan11/qjx/gradient_experiment/transE_checkpoint/transe.ckpt')

Finish initializing...


Epoch 9 | loss: 7.714484: 100%|██████████| 10/10 [00:14<00:00,  1.48s/it] 


In [8]:
transe.load_checkpoint('/home/zixuan11/qjx/gradient_experiment/transE_checkpoint/transe.ckpt')

In [9]:
transe

TransE(
  (ent_embeddings): Embedding(14541, 200)
  (rel_embeddings): Embedding(237, 200)
)

In [10]:
device = torch.device('cuda:5' if torch.cuda.is_available() else "cpu")

In [11]:
transe.ent_embeddings(torch.tensor(0).to(device))

tensor([ 1.9306e-02, -2.5632e-02, -1.3782e-02,  2.5832e-02,  9.6677e-03,
         4.7554e-03, -6.5929e-03, -4.0559e-03,  1.4569e-02, -1.9582e-02,
         2.8162e-02, -5.5469e-03, -3.5969e-03,  2.2285e-02, -8.8358e-03,
         6.2256e-04, -2.6435e-02, -2.4656e-02,  1.3898e-02,  1.9853e-03,
        -1.7810e-02,  1.2797e-02,  8.4199e-03,  1.6161e-02,  3.2635e-03,
         4.9217e-03,  2.8816e-02, -1.0700e-02, -9.0739e-03,  2.6620e-02,
        -3.8206e-03, -1.8923e-02,  2.2267e-02,  4.6751e-04, -5.9256e-03,
         1.7030e-02,  3.6299e-02,  1.0356e-02, -2.3866e-02, -9.3976e-03,
         1.5638e-02,  1.5685e-02,  1.8958e-02, -2.2883e-02, -4.3815e-03,
         2.0357e-02, -2.6899e-03,  1.6311e-02,  1.3618e-02, -1.1739e-02,
         1.6477e-02,  1.0733e-03, -5.3514e-03,  5.6657e-03, -1.7748e-02,
         6.0230e-03, -4.9133e-04, -2.4723e-02, -4.4542e-03, -5.6122e-03,
         2.0770e-03, -8.7193e-03, -1.7554e-02,  5.3800e-03, -5.1983e-03,
         4.6893e-03, -6.6509e-03,  2.8260e-02, -8.3

In [13]:
tester = Tester(model = transe, data_loader = test_dataloader, use_gpu = True)
tester.run_link_prediction(type_constrain = False)

100%|██████████| 20466/20466 [00:25<00:00, 788.27it/s]

no type constraint results:
metric:			 MRR 		 MR 		 hit@10 	 hit@3  	 hit@1 
l(raw):			 0.079224 	 737.892517 	 0.184306 	 0.076957 	 0.028926 
r(raw):			 0.225517 	 263.292938 	 0.415909 	 0.257891 	 0.127773 
averaged(raw):		 0.152370 	 500.592712 	 0.300108 	 0.167424 	 0.078349 

l(filter):		 0.152138 	 506.509247 	 0.292485 	 0.171602 	 0.080573 
r(filter):		 0.328030 	 240.873108 	 0.536890 	 0.388303 	 0.214502 
averaged(filter):	 0.240084 	 373.691162 	 0.414688 	 0.279952 	 0.147537 
0.41468775272369385
0.414688


(0.24008432030677795,
 373.691162109375,
 0.41468775272369385,
 0.27995210886001587,
 0.14753738045692444)